<a href="https://colab.research.google.com/github/dkapitan/scik/blob/master/notebooks/ggd-rijnmond-rotterdam-covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# make sure we're using pandas >= 1.0 and we need CBS OData
!pip install update pandas cbsodata

In [0]:
import requests
from zipfile import ZipFile
from io import BytesIO, StringIO

from bs4 import BeautifulSoup
import pandas as pd
import cbsodata


def get_gwb():
    """
    Retrieves data from CBS for mapping each address to buurt, wijk and gemeente from 2019.

    Returns:
      list of length 4 with dataframes [buurt, gemeente, mapping address --> gwb, wijken]

    Source: https://www.cbs.nl/nl-nl/maatwerk/2019/42/buurt-wijk-en-gemeente-2019-voor-postcode-huisnummer
    Numeric columns are downcasted to uint16 to lower memory footprint
    """
    URL = 'https://www.cbs.nl/-/media/_excel/2019/42/2019-cbs-pc6huisnr20190801_buurt.zip'
    get_ = requests.get(URL)
    with ZipFile(BytesIO(get_.content)) as zipfile:
      dfs = [pd.read_csv(zipfile.open(file),
                          delimiter=';',
                          )
              for file in zipfile.namelist()
             ]
    for df in dfs:
      df.loc[:, df.select_dtypes('int64').columns] = df.select_dtypes('int64').astype('uint16')
    return dfs


def get_kerncijfers_gwb():
  """
  Retrieves Kerncijfers Wijken en Buurten from 2019
  """
  URL = 'https://www.cbs.nl/-/media/cbs/dossiers/nederland%20regionaal/wijk-en-buurtstatistieken/_exel/kwb-2019.xls'
  return pd.read_excel(URL)


def get_rivm_data():
    """
    Scrapes RIVM website for number of reported hospitalizations per gemeente.
    """
    RIVM = 'https://www.rivm.nl/actuele-informatie-over-coronavirus'
    rivm = requests.get(RIVM)
    soup = BeautifulSoup(rivm.content, 'html.parser')
    data = StringIO(soup.find(id='csvData').text)
    t = soup.find(class_='content-date-edited')
    df = pd.read_csv(data, sep=';', skip_blank_lines=1)
    df['timestamp_rivm'] = t.contents[0]
    return df


def get_regionale_indelingen():
  """
  Retrieves regionale indelingen from CBS.

  https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=84721NED&_theme=232
  """
  return pd.DataFrame(cbsodata.get_data('84721NED'))


def get_data_ggd_rijnmond():
  """ Retrieves data from GGD Rijnmond on number of positive tests"""
  URL = 'https://www.ggdrotterdamrijnmond.nl/nieuws/patienten-coronavirus-in/'
  df = pd.read_html(URL, header=0)[0].iloc[0:-1,:]
  df.iloc[:,-1] = pd.to_numeric(df.iloc[:,-1], downcast='unsigned')
  s = df.columns[-1]
  rename = {k: v for k,v in zip(df.columns, ('gemeente', 'aantal_positief_getest'))}
  df['timestamp_ggd'] = s[s.find("(")+1:s.find(")")]
  return df.rename(columns=rename)

In [0]:
# load all data
buurt, gemeente, adres, wijk = get_gwb()
kerncijfers = get_kerncijfers_gwb()
kerncijfers_gemeente = kerncijfers.loc[kerncijfers.recs == 'Gemeente', :]
rivm = get_rivm_data()
gebieden = get_regionale_indelingen()
ggd010 = get_data_ggd_rijnmond()

In [28]:
df = (adres[['Gemeente2019', 'Wijk2019']]
      .drop_duplicates()
      .astype('int32')
      .join(gemeente.set_index('Gemcode2019'), on='Gemeente2019')
      .join(wijk.set_index('Wijkcode2019'), on='Wijk2019')
)
df

,Gemeente2019,Wijk2019,Gemeentenaam2019,Wijknaam_2019K_NAAM
0,363,36304,Amsterdam,Nieuwmarkt/Lastage
3773,363,36309,Amsterdam,Oostelijke Eilanden/Kadijken
3933,363,36301,Amsterdam,Burgwallen-Nieuwe Zijde
3989,363,36300,Amsterdam,Burgwallen-Oude Zijde
8261,363,36305,Amsterdam,Haarlemmerbuurt
...,...,...,...,...
7707760,1966,65533,Het Hogeland,Wijk 05
7707828,1966,65532,Het Hogeland,Wijk 04
7709168,1966,65535,Het Hogeland,Wijk 07
7713352,1966,0,Het Hogeland,Wijk 08


In [4]:
rivm

,Gemnr,Gemeente,Meldingen,Zkh opname,Overleden,BevAant,Meldingen per 100.000,Zkh opname per 100.000,Overleden per 100.000,timestamp_rivm
0,796,'s-Hertogenbosch,323,111,52,155113,208.2,71.6,33.5,Wijzigingsdatum 16-04-2020 | 14:19
1,1680,Aa en Hunze,11,1,0,25453,43.2,3.9,0.0,Wijzigingsdatum 16-04-2020 | 14:19
2,358,Aalsmeer,31,7,2,31857,97.3,22.0,6.3,Wijzigingsdatum 16-04-2020 | 14:19
3,197,Aalten,30,2,3,27117,110.6,7.4,11.1,Wijzigingsdatum 16-04-2020 | 14:19
4,59,Achtkarspelen,7,0,0,27836,25.1,0.0,0.0,Wijzigingsdatum 16-04-2020 | 14:19
...,...,...,...,...,...,...,...,...,...,...
350,879,Zundert,49,19,8,21821,224.6,87.1,36.7,Wijzigingsdatum 16-04-2020 | 14:19
351,301,Zutphen,46,18,8,47942,95.9,37.5,16.7,Wijzigingsdatum 16-04-2020 | 14:19
352,1896,Zwartewaterland,107,31,12,22690,471.6,136.6,52.9,Wijzigingsdatum 16-04-2020 | 14:19
353,642,Zwijndrecht,74,13,4,44741,165.4,29.1,8.9,Wijzigingsdatum 16-04-2020 | 14:19


In [5]:
# data GGD Rotterdam Rijnmond
df = (ggd010
      .replace({'Capelle a/d IJssel': 'Capelle aan den IJssel', 'Krimpen a/d IJssel': 'Krimpen aan den IJssel'})
      .join(rivm.set_index('Gemeente'), 'gemeente')
      .rename(columns={'Aantal': 'aantal_ziekenhuisopnames',
                       'BevAant': 'aantal_inwoners',
                       'Gemnr': 'gemeente_code',
                       'gemeente': 'gemeente_naam',
                       'Aantal per 100.000 inwoners': 'prevalentie_ziekenhuisopnames'})
      .assign(prevalentie_positief_getest=lambda df: (df.aantal_positief_getest/df.aantal_inwoners*10**5).astype('uint8'))
      .loc[:, ['gemeente_code',
          'gemeente_naam',
          'aantal_inwoners',
          'aantal_positief_getest',
          'aantal_ziekenhuisopnames',
          'prevalentie_positief_getest',
          'prevalentie_ziekenhuisopnames',
              'timestamp_ggd',
              'timestamp_rivm']]
      .sort_values('gemeente_naam')
      .assign(gemeente_code=lambda df: 'GM' + df.gemeente_code.astype(str).str.pad(4, 'left', '0'))
)
with open('../data/ggd-rivm.csv', 'w') as file:
    df.to_csv(file, index=False)

KeyError: ignored

In [0]:
# indeling gemeentes naar GGD regios
gem_ggd_cols = ['Code_1', 'RegioS', 'Code_14', 'Naam_15']
gem_ggd_rename = {k:v for k,v in zip(gem_ggd_cols, ['gemeente_code', 'gemeente_naam', 'ggd_code', 'ggd_naam'])}
gem_ggd = (gebieden
     .loc[:,gem_ggd_cols]
     .drop_duplicates()
     .apply(lambda x: x.str.strip())
     .rename(columns=gem_ggd_rename)
    )
with open('../data/gemeente-ggd.csv', 'w') as file:
    gem_ggd.to_csv(file, index=False)

In [0]:
# indeling PC4 naar gemeente
adres['pc4'] = adres.PC6.str[0:4]
pc4_gem = (adres
           .loc[:,['pc4', 'Gemeente2019']]
           .drop_duplicates()
           .astype({'Gemeente2019': str})
           .assign(gemeente_code=lambda df: 'GM' + df.Gemeente2019.str.pad(4, 'left', '0'))
           .join(gem_ggd.set_index('gemeente_code'), on='gemeente_code')
)
with open('../data/pc4-gemeente-ggd.csv', 'w') as file:
    pc4_gem.to_csv(file, index=False)